# YOLO Model Validation and Results Export

This notebook demonstrates how to validate a YOLO model and export the validation results in multiple formats including CSV, HTML, XML, SQL, and JSON. This is particularly useful for:

- **Model Performance Analysis**: Get detailed metrics for each class
- **Data Integration**: Export results to databases or other analysis tools
- **Reporting**: Generate formatted reports for stakeholders
- **Further Processing**: Use results in downstream analysis pipelines

## Key Features Covered:
- Model validation with YOLO11
- Exporting validation metrics in multiple formats
- Confusion matrix analysis and export
- Best practices for result handling

## Environment Setup and Dependencies

First, we'll install the Ultralytics library which provides the YOLO implementation and check our environment setup. This ensures we have all necessary dependencies for model validation and result export.

In [1]:
# Install Ultralytics YOLO package
%pip install ultralytics -q

# Import and verify installation
import ultralytics
ultralytics.checks()

print("\n📦 Dependencies installed successfully!")
print("✅ Ready to proceed with model validation")

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.5/107.7 GB disk)


## Model Loading and Validation

Here we load a pre-trained YOLO11 nano model and validate it on the COCO8 dataset. The validation process evaluates the model's performance across different object classes and generates comprehensive metrics including:

- **Precision**: Accuracy of positive predictions
- **Recall**: Ability to find all positive instances
- **F1-Score**: Harmonic mean of precision and recall
- **mAP (mean Average Precision)**: Overall detection accuracy
- **Confusion Matrix**: Detailed classification results

💡 **Note**: Replace `"coco8.yaml"` with your custom dataset configuration file if using a custom-trained model.

In [ ]:
from ultralytics import YOLO
import os

# Load pre-trained YOLO11 nano model
print("🔄 Loading YOLO11 nano model...")
model = YOLO("yolo11n.pt")

print("🔍 Starting model validation...")
print("📊 This may take a few minutes depending on dataset size...\n")

# Validate the model on COCO8 dataset
# For custom models, replace "coco8.yaml" with your data.yaml file
metrics = model.val(data="coco8.yaml", verbose=True)

print("\n✅ Validation completed successfully!")
print(f"📈 Overall mAP@0.5: {metrics.box.map50:.4f}")
print(f"📈 Overall mAP@0.5:0.95: {metrics.box.map:.4f}")

## Validation Results Export

The YOLO metrics object provides several convenient methods to export validation results in different formats. This section demonstrates how to export the same data in various formats for different use cases:

- **DataFrame**: For immediate analysis in Python/Jupyter
- **CSV**: For spreadsheet applications (Excel, Google Sheets)
- **SQL**: For database integration and queries
- **HTML**: For web display and reporting
- **XML**: For structured data exchange
- **JSON**: For API integration and web applications

### 📊 Pandas DataFrame Format

The DataFrame format is perfect for immediate analysis within Python. It provides a structured view of per-class performance metrics that can be easily manipulated, filtered, and analyzed.

In [12]:
# Display results as pandas DataFrame
df_results = metrics.to_df()
print("📋 Validation Results (DataFrame Format):")
print("=" * 80)
print(df_results)

# Additional analysis examples
print("\n📊 Quick Statistics:")
print(f"   • Number of classes evaluated: {len(df_results)}")
print(f"   • Best performing class (by mAP): {df_results.loc[df_results['box-map'].idxmax(), 'class_name']}")
print(f"   • Lowest performing class (by mAP): {df_results.loc[df_results['box-map'].idxmin(), 'class_name']}")
print(f"   • Average precision across all classes: {df_results['box-p'].mean():.4f}")
print(f"   • Average recall across all classes: {df_results['box-r'].mean():.4f}")

     class_name    box-p  box-r   box-f1  box-map  box-map50  box-map75
0        person  0.55725    0.6  0.57783  0.63162    0.84684     0.6918
1           dog  0.54802    1.0  0.70803  0.63162    0.84684     0.6918
2         horse  0.53058    1.0  0.69331  0.63162    0.84684     0.6918
3      elephant  0.37078    0.5  0.42580  0.63162    0.84684     0.6918
4      umbrella  0.56900    1.0  0.72531  0.63162    0.84684     0.6918
5  potted plant  0.84718    1.0  0.91727  0.63162    0.84684     0.6918


### 📄 CSV (Comma Separated Values) Format

CSV format is ideal for:
- Opening in spreadsheet applications (Excel, Google Sheets)
- Data exchange between different systems
- Simple data storage and backup
- Integration with data analysis tools

In [13]:
# Export to CSV format
csv_results = metrics.to_csv()
print("📄 Validation Results (CSV Format):")
print("=" * 80)
print(csv_results)

# Optionally save to file
csv_filename = "validation_results.csv"
with open(csv_filename, 'w') as f:
    f.write(csv_results)
    
print(f"💾 Results saved to: {csv_filename}")
print("📁 You can download this file for use in Excel or other applications")

,class_name,box-p,box-r,box-f1,box-map,box-map50,box-map75
0,person,0.55725,0.6,0.57783,0.63162,0.84684,0.6918
1,dog,0.54802,1.0,0.70803,0.63162,0.84684,0.6918
2,horse,0.53058,1.0,0.69331,0.63162,0.84684,0.6918
3,elephant,0.37078,0.5,0.4258,0.63162,0.84684,0.6918
4,umbrella,0.569,1.0,0.72531,0.63162,0.84684,0.6918
5,potted plant,0.84718,1.0,0.91727,0.63162,0.84684,0.6918



### 🗄️ SQL Database Format

SQL export creates a SQLite database file that can be:
- Queried using SQL commands
- Integrated into database-driven applications
- Used for complex data analysis and reporting
- Connected to business intelligence tools

💡 **Tip**: To view the SQL file, open the [SQLite Viewer](https://inloop.github.io/sqlite-viewer/) and upload the `results.db` file.

In [14]:
# Export to SQL database
print("🗄️ Exporting to SQL database...")
metrics.to_sql()

# Verify the database file was created
if os.path.exists('results.db'):
    file_size = os.path.getsize('results.db')
    print(f"✅ SQL database created successfully!")
    print(f"📁 File: results.db ({file_size} bytes)")
    print(f"🔍 View online: https://inloop.github.io/sqlite-viewer/")
    
    # Show some SQL query examples
    print("\n📝 Example SQL queries you can run:")
    print("   • SELECT * FROM results WHERE [box-map] > 0.7")
    print("   • SELECT class_name, [box-p], [box-r] FROM results ORDER BY [box-f1] DESC")
    print("   • SELECT AVG([box-map]) as avg_map FROM results")

Results saved to SQL table 'results' in 'results.db'.


### 🌐 Additional Export Formats

YOLO also supports exporting to HTML, XML, and JSON formats. Each format serves different purposes:

- **HTML**: Perfect for web reports and dashboards
- **XML**: Structured data exchange, especially in enterprise environments
- **JSON**: API integration, web applications, and modern data pipelines

In [ ]:
# Export to HTML format
print("🌐 Exporting to HTML format...")
html_results = metrics.to_html()
with open('validation_results.html', 'w') as f:
    f.write(html_results)
print("✅ HTML file created: validation_results.html")

# Export to XML format
print("\n📄 Exporting to XML format...")
xml_results = metrics.to_xml()
with open('validation_results.xml', 'w') as f:
    f.write(xml_results)
print("✅ XML file created: validation_results.xml")

# Export to JSON format
print("\n🔧 Exporting to JSON format...")
json_results = metrics.to_json()
with open('validation_results.json', 'w') as f:
    f.write(json_results)
print("✅ JSON file created: validation_results.json")

print("\n🎉 All export formats completed successfully!")
print("📁 Files created: .csv, .html, .xml, .json, .db")

## 🎯 Confusion Matrix Analysis and Export

The confusion matrix provides detailed insights into model performance by showing:
- **True vs Predicted classifications** for each class
- **Misclassification patterns** between similar classes  
- **Background vs object detection** accuracy
- **Class-specific strengths and weaknesses**

This is particularly valuable for:
- Identifying which classes are often confused with each other
- Understanding false positive and false negative patterns
- Deciding on model improvement strategies
- Creating targeted training data augmentation

💡 **Note**: The confusion matrix can also be exported to the same formats as the validation metrics.

In [11]:
# Display confusion matrix as DataFrame
print("🎯 Confusion Matrix Analysis:")
print("=" * 80)
cm_df = metrics.confusion_matrix.to_df()
print(f"📊 Matrix dimensions: {cm_df.shape[0]}x{cm_df.shape[1]}")
print(f"📈 Total predictions analyzed: {cm_df.select_dtypes(include=['number']).sum().sum():.0f}")
print("\nConfusion Matrix (showing first 10 rows for readability):")
print(cm_df.head(10))

# Export confusion matrix in different formats
print("\n💾 Exporting confusion matrix...")

# CSV format
cm_csv = metrics.confusion_matrix.to_csv()
with open('confusion_matrix.csv', 'w') as f:
    f.write(cm_csv)
print("✅ Confusion matrix CSV: confusion_matrix.csv")

# SQL format 
metrics.confusion_matrix.to_sql()
print("✅ Confusion matrix added to SQL database")

# Other formats
formats = ['html', 'xml', 'json']
for fmt in formats:
    method = getattr(metrics.confusion_matrix, f'to_{fmt}')
    result = method()
    filename = f'confusion_matrix.{fmt}'
    with open(filename, 'w') as f:
        f.write(result)
    print(f"✅ Confusion matrix {fmt.upper()}: {filename}")

print("\n🎉 Confusion matrix exported in all formats!")

     Predicted  person  bicycle  car  motorcycle  airplane  bus  train  truck  \
0       person     4.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
1      bicycle     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
2          car     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
3   motorcycle     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
4     airplane     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
..         ...     ...      ...  ...         ...       ...  ...    ...    ...   
76    scissors     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
77  teddy_bear     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
78  hair_drier     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
79  toothbrush     0.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   
80  background     6.0      0.0  0.0         0.0       0.0  0.0    0.0    0.0   

    boat  ...  sink  refrig

## 📋 Summary and Next Steps

This notebook demonstrated how to:

✅ **Load and validate** a YOLO model with comprehensive metrics  
✅ **Export validation results** in multiple formats (CSV, HTML, XML, JSON, SQL)  
✅ **Analyze confusion matrices** for detailed performance insights  
✅ **Save results** for integration with other tools and workflows  

### 🔄 Potential Next Steps:

1. **Custom Dataset Validation**: Replace `"coco8.yaml"` with your own dataset configuration
2. **Automated Reporting**: Set up scripts to automatically generate and send validation reports
3. **Performance Monitoring**: Track model performance over time using the exported data
4. **Model Comparison**: Use these exports to compare different model versions or architectures
5. **Integration**: Connect the SQL database to your BI tools or monitoring dashboards

### 📚 Additional Resources:
- [Ultralytics Documentation](https://docs.ultralytics.com/)
- [YOLO Model Hub](https://github.com/ultralytics/ultralytics)
- [SQLite Viewer](https://inloop.github.io/sqlite-viewer/) for viewing database exports

---
*Happy validating! 🚀*